
# Quiver Plots & Movies

In [ ]:
import sys
if '..' not in sys.path:
    sys.path.append('..')

from pathlib import Path
import subprocess

import matplotlib.pyplot as plt
import cmocean
from tqdm.auto import tqdm
from IPython.display import Video


import dustpy
import dustpy.constants as c
from plot_helper import plot_quiver, read_paletton_text, plot_size_distri, get_transparent_cmap
import helper

# Read the paletton color file

colors = (read_paletton_text()[[3,0,2,1], 0, :]/255).tolist()
plt.imshow([colors]).axes.axis('off');

In [ ]:
reader = dustpy.hdf5writer()
data_dir = Path('~').expanduser() / 'DATA/araa_data/lowres_highvf/data_bump'
# data_dir = Path('data_bump')
reader.datadir = str(data_dir)
files = reader.read.listfiles()
times = reader.read.sequence('t') / c.year
file = Path(files[np.abs(times - 1e6).argmin()])

sim = reader.read.output(str(file))

## Make a PDF

In [ ]:
f, axs = plt.subplots(1, 2, figsize=(13, 4), gridspec_kw={'wspace':0.2});

ax = plot_size_distri([str(file) for file in files],
                      radii_au=[3, 30, 100],
                      times=[1e4, 1e5, 1e6, 2e6, 3e6],
                      legend=True,
                      cols=colors, figsize=(6.4, 4), ax=axs[1])
ax.set_facecolor('none')
helper.apply_araa_style(ax)
#ax.figure.savefig(helper.output_dir / f'size_distribution_{Path(reader.datadir).name}.pdf', transparent=True, bbox_inches='tight')

output_file = helper.output_dir / file.with_suffix('.pdf').name.replace('data', 'frame_' + file.parent.name.replace('data_', '') + '_')
f, ax, sols = plot_quiver(str(file), vmin=1e-8, cmap='cmo.algae', cols=colors, figsize=(6.4, 4), ax=axs[0])
ax.set_facecolor('none')
ax.set_xlim(right=300)
helper.apply_araa_style(ax)

f.dpi=150
f.savefig(output_file, transparent=True, bbox_inches='tight')

In [ ]:
!open {helper.output_dir}

## Make the frames & compile movie

In [ ]:
dirname = Path('frames')
dirname.mkdir(exist_ok=True)

for i, file in tqdm(enumerate(files), total=len(files)):
    f, ax, sols = plot_quiver(file, vmin=1e-7,
                              #cmap=get_transparent_cmap('cmo.algae'),
                              cmap='cmo.algae',
                              cols=colors, figsize=(6.4, 4))
    ax.set_xlim(right=300)
    helper.apply_araa_style(ax)
    f.savefig(dirname / f'frame_{i:03d}.jpg', transparent=False, dpi=300)

    #ax.set_facecolor('none')
    #f.savefig(dirname / f'frame_{i:03d}.png', transparent=True, dpi=300)
    
    plt.close(f)

Transparent

In [ ]:
ret = subprocess.check_output(
    f'ffmpeg -y -r 12 -i {dirname}/frame_%03d.png -c:v prores_ks -profile:v 5 -vendor apl0  -bits_per_mb 8000 -pix_fmt yuva444p10le out_t.mov'.split(),
    stderr=subprocess.STDOUT)

Non-Transparent

In [ ]:
ret = subprocess.check_output(
    f'ffmpeg -y -framerate 7 -i {dirname}/frame_%03d.jpg -c:v libx264 -crf 23 -pix_fmt yuv420p out.mp4'.split(),
    stderr=subprocess.STDOUT)

In [ ]:
Video('out.mp4', width=500)

## Test Bouncing barrier

In [ ]:
from matplotlib.colors import LogNorm
ab = (3 * 3.3e-3 / (4*np.pi*sim.dust.rhos.mean()))**(1/3)
a_bounce = ab * (2 / (3 * np.pi) * sim.gas.Sigma / (sim.gas.alpha * ab * sim.dust.rhos.mean() * sim.gas.cs**2))**(18/48)

A = np.mean(sim.grid.m[1:] / sim.grid.m[:-1])
B = 2 * (A - 1) / (A + 1)

f, ax = plt.subplots()
ax.contourf(sim.grid.r / c.au, sim.dust.a[0], sim.dust.Sigma.T / B,
            norm=LogNorm(1e-8, 1e1),
            levels=10.**np.arange(-8, 2), cmap=get_transparent_cmap('cmo.algae'))
ax.loglog(sim.grid.r / c.au, a_bounce)
ax.loglog(sim.grid.r / c.au, 1e-3 * (sim.grid.r / (10*c.au))**(-9/48))

# Mass evolution plot

In [ ]:
data_dirs = list((Path('~').expanduser() / 'DATA/araa_data/lowres_highvf').glob('data_*'))

In [ ]:
def plot_blue_green_transition(ax):
    _x = np.linspace(0, 1, 100)
    _b = 1 / (np.exp((_x - 0.47) / 0.05) + 1)
    _g = 1 - 1 / (np.exp((_x - 0.53) / 0.05) + 1)
    
    img1 = plt.get_cmap('Blues')(1-_x)
    img2 = plt.get_cmap('Greens')(_x)
    
    img1[:, -1] = _b
    img2[:, -1] = _g
    
    ax.imshow([img1], aspect='auto', alpha=0.75)
    ax.imshow([img2], aspect='auto', alpha=0.75)
    ax.axis('off')
    ax.set_facecolor('none')

In [ ]:
f, ax = plt.subplots(figsize=(6.4, 4))
ax.loglog()
xlim = [1e3, 3e6]
ylim = [5e-3, 2]

ax.set_xlim(xlim)
ax.set_ylim(ylim)

# get conversion functions

axis_to_data = ax.transAxes + ax.transData.inverted()
data_to_axis = axis_to_data.inverted()#

# plot the lines

for i, data_dir in enumerate(data_dirs[::-1]):

    writer = dustpy.hdf5writer()
    writer.datadir = str(data_dir)
    
    data = writer.read.output(0)
    r = data.grid.r
    A = data.grid.A
    
    time = writer.read.sequence('t')
    sigg = writer.read.sequence('gas.Sigma')
    sigd = writer.read.sequence('dust.Sigma')
    
    Mg = (data.grid.A * sigg).sum(-1)
    Md = (data.grid.A * sigd.sum(-1)).sum(-1)
    
    l1, = ax.loglog(time / c.year, Mg / Mg[0], c=colors[i+1], ls='--')
    l2, = ax.loglog(time / c.year, Md / Md[0], c=colors[i+1], ls='-')

    # Compute dust mass outside planet and display it
    if ('_bump' in str(data_dir)):
        i_p = r.searchsorted(data.planets.jupiter.a[0])
        m_out = (data.dust.Sigma.sum(-1) * data.grid.A)[i_p:].sum()
        _ = ax.axhline(m_out / Md[0], c='0.35', ls='--', zorder=1)
        ax.text(axis_to_data.transform([0.2, 0])[0], m_out / Md[0],
                r'dust mass at $r>r_\mathrm{gap}$',
                ha='center', va='bottom', fontsize=8, c=_.get_color())

ax.set_xlabel('time [yr]')
ax.set_ylabel('mass / initial mass')

ax.loglog([], c=colors[2], label='with pressure bump')
ax.loglog([], c=colors[1], label='without pressure bump')
ax.loglog([], c='k', ls='--', label='dust')
ax.loglog([], c='k', ls='-', label='gas')
ax.loglog(time / c.year, 0.3 * ((time + 1e-100) / (1e5 * c.year))**-1, c='k', ls=':')
ax.text(0.9, 0.25, r'$\propto t^{-1}$',
        ha='center', va='bottom', transform=ax.transAxes,
        rotation=-45, transform_rotates_text=True)

# compute and show time scale

Ng = 4
Rg = 100
tgrow = np.interp(Rg * c.au, r, Ng / (data.grid.OmegaK * data.dust.eps))
ax.axvline(tgrow / c.year, c='k', ls='-.')
ax.text(data_to_axis.transform([tgrow / c.year, 0])[0], 0.5,
        fr'${Ng} \times t_\mathrm{{grow}} \mathsf{{~at~}} {Rg}$ au',
        va='center', ha='right', rotation=90, transform=ax.transAxes)

ax.legend().get_frame().set_alpha(0)

#### background coloring ####

fill = [xlim[0], tgrow / c.year]
y0 = 1.3

# first fill is normal
ax.fill_between(fill, ylim[0] * np.ones(2), y2=ylim[1] * np.ones(2),
                color=plt.get_cmap('Blues')(1.0),
                alpha=0.5, ec='none', zorder=0)

# add another axes to overlay an image
_x0, _y0 = data_to_axis.transform([tgrow / c.year, ylim[0]])
_x1, _y1 = data_to_axis.transform([xlim[1], ylim[1]])
axi = ax.inset_axes([_x0, _y0, _x1 - _x0, _y1 - _y0])
plot_blue_green_transition(axi)
axi.zorder=0

ax.set_xlim(xlim)
ax.set_ylim(ylim)    

# text for the 3 epochs

opts = dict(ha='center', weight='bold', fontsize=8)
ax.text(10.**np.mean(np.log10(fill)), y0, 'initial growth', **opts)
ax.text(1.5e5, y0, 'frag. limited', **opts)
ax.text(1.3e6, y0, 'drift limited', **opts)

output_file = helper.output_dir / ('mass_evolution_' + data_dir.parent.with_suffix('.pdf').name)
ax.set_facecolor('none')
helper.apply_araa_style(ax)
f.savefig(output_file)

In [ ]:
m_out_t = (sigd.sum(-1) * data.grid.A[None, ...])[:, i_p:].sum(-1)